In [2]:
import os,csv,pandas,torch,torchvision
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from torch.utils.data import DataLoader,TensorDataset
import torchvision.models as models
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import CrossEntropyLoss
import torch.optim as optim
import torch.nn.functional as F

Using TensorFlow backend.
/home/ranjith.reddy/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ranjith.reddy/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ranjith.reddy/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ranjith.reddy/.local/lib/python3.7/sit

In [3]:
!nvidia-smi

Mon Apr 20 02:43:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:02:00.0 Off |                  N/A |
| 38%   41C    P0    39W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
path="/ssd_scratch/cvit/ranjith"
x_train_path = os.path.join(path,'train.csv')
x_test_path = os.path.join(path,'test.csv')
train_data = pandas.read_csv(x_train_path)
gt = train_data['label']
labels_data = gt.to_numpy()
labels = labels_data
test_data = pandas.read_csv(x_test_path)
df_train = train_data.drop('label', 1)
data = df_train.to_numpy()
x_test = test_data.to_numpy()
data = data.reshape(42000,28,28)
x_test = x_test.reshape(28000,28,28)
x_train = data.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255
X_train,X_dev,Y_train,Y_dev= train_test_split(x_train,labels,test_size=0.2)
Xtrain = torch.Tensor(X_train)
Xdev = torch.Tensor(X_dev)
Ytrain = torch.Tensor(Y_train)
Ydev = torch.Tensor(Y_dev)
Xtest = torch.Tensor(x_test)
Xtrain = torch.stack([Xtrain, Xtrain, Xtrain], dim=1)
Xdev = torch.stack([Xdev, Xdev, Xdev], dim=1)
Xtest = torch.stack([Xtest, Xtest, Xtest], dim=1)

In [6]:
train_tensor = TensorDataset(Xtrain,Ytrain)
val_tensor = TensorDataset(Xdev,Ydev)
test_tensor = TensorDataset(Xtest)

In [7]:
def get_train_loader(batch_size=64):
    train_loader = DataLoader(train_tensor,batch_size,shuffle=True)
    return train_loader
val_loader = DataLoader(val_tensor,batch_size=64)
test_loader = DataLoader(test_tensor,batch_size=1)

In [8]:
resnet18=models.resnet18(pretrained=True)
resnet18.fc.out_features=10
resnet18.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
def createLossandOptimizer(net,lr=0.001):
    loss = CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(),lr,weight_decay=1e-6)
    return (loss,optimizer)

In [ ]:
def train_model(net,device,epochs,batchSize=64,learningrate=0.001):
    for epoch in range(epochs):
        train_loader = get_train_loader(batchSize)
        loss,optimizer = createLossandOptimizer(net,lr=learningrate)
        net.train()
        print("started epoch = ,",epoch)
        for i,data in enumerate(train_loader):
            inp,label = data
            X_train,Y_train = Variable(inp).to(device),Variable(label).to(device,dtype=torch.long)
            optimizer.zero_grad() #set input gradients to zero, generally gradients are not replaced 
            
            #Forward pass,Backward pass,optimize
            outputs = net(X_train)
            outputs.to(device)
            #print(Y_train.shape,outputs.shape)
            loss_size = loss(outputs,Y_train)
            loss_size.backward()
            optimizer.step()
        # evaluation for batch, calculating loss on validation data 
        net.eval()
        tot_val_loss = 0
        for i,data in enumerate(val_loader):
            inp,label = data
            X_train = Variable(inp).to(device)
            Y_train = Variable(label).to(device,dtype=torch.long)
            valid_net = net.eval()
            outputs = valid_net(X_train)
            outputs.to(device)
            loss_size = loss(outputs,Y_train)
            tot_val_loss += loss_size.item()
        print("Validation loss = {:.2f} ".format(tot_val_loss))

In [ ]:
train_model(resnet18,device,30,128,learningrate=0.001)

In [ ]:
torch.save(resnet18.state_dict(), "final_resnet_model.pt")

In [9]:
resnet18.load_state_dict(torch.load('final_resnet_model.pt'))

<All keys matched successfully>

In [16]:
count=0
resnet18.eval()
final_results=[]
with open("output_pytorch.csv",'w') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(["ImageId","Label"])
    for i,data in enumerate(test_loader):
        inputs = data[0]
        inputs = Variable(inputs).to(device)
        outputs = resnet18(inputs)
        _, predicted = torch.max(outputs.data, 1)
        csvwriter.writerow([i+1,predicted[0].item()])

In [17]:
out_path = os.path.join(".",'output_pytorch.csv')
train_data = pandas.read_csv(out_path)
train_data

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
